In [ ]:
import pandas as pd
import textblob
from textblob import TextBlob
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import nltk
from datasets import load_dataset
nltk.download('wordnet')
nltk.download('wordnet_ic')
import numpy as np
import string
import pandas as pd
from sentence_transformers import SentenceTransformer
from top2vec import Top2Vec
import pandas as pd
from sklearn.cluster import DBSCAN

#model = Top2Vec(documents)

In [ ]:
def find_most_common_words(skDocsTfIdfdf, threshold):
    # Calculate the sums of the tf-idf values across all the documents
    tf_idf_sums = skDocsTfIdfdf.sum(axis=0)
    
    # Sort the tf-idf values in descending order
    sorted_tf_idf_sums = tf_idf_sums.sort_values(ascending=False)
    
    # Filter out the words with the highest tf-idf values, so that only the top 50% of values are displayed
    filtered_tf_idf_sums = sorted_tf_idf_sums[sorted_tf_idf_sums > sorted_tf_idf_sums.quantile(1-threshold)]
    return(filtered_tf_idf_sums)
    # Return the sorted dataframe

    
def createVocab_lemma(docs):
    vocab={}
    for doc in docs:
        doc= doc.translate(str.maketrans('', '', string.punctuation))
        for word in word_tokenize(doc.lower()):
            if word not in vocab.keys():
                vocab[word] = 1
                # Create a synonym list for each word
                word_synsets = [wordnet.synsets(word) for word in word_tokenize(doc.lower())]
                synonyms = []
                for word_synset in word_synsets:
                    if (word_synset):
                        for lemma in word_synset[0].lemmas():
                            synonyms.append(lemma.name())
                vocab[word] = synonyms
    return vocab

In [ ]:

dataset = load_dataset("squad_v2")

df = pd.DataFrame(dataset['train'])

contexts_df = pd.DataFrame(np.unique(df['context']),columns=['context'])
contexts_df.to_csv('contexts.csv')

questions_df = df[['question']]
answers_df = pd.DataFrame(['' if len(d['text'])==0 else d['text'][0] for d in df['answers']])
answers_df.columns = ['answer']
qa_df = pd.concat([questions_df,answers_df],axis=1)
qa_df.to_csv('qa.csv')



In [ ]:

model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:

df = pd.read_csv('qa.csv',index_col=0)
#df = pd.read_csv('qa.csv',index_col=0)

docs = pd.DataFrame(np.unique(df['question']),columns=['question'])['question'][0:250]
#docs = pd.DataFrame(np.unique(df['context']),columns=['context'])['context'][0:100]
tokens_ = [docs.iloc[c].split(' ') for c in np.array(range(0,len(docs),1))]
token_lengths = [len(t) for t in tokens_]

In [ ]:
questions = np.random.choice(df['question'].values, 250, replace=False)
encodings = [model.encode(q) for q in questions]

In [ ]:
from sentence_transformers import SentenceTransformer, util
cosine_scores = util.cos_sim(encodings, encodings)
print(type(cosine_scores))
print(cosine_scores)

In [ ]:
def derive_centroids(data, labels):
    # Calculate number of clusters
    n_clusters = len(np.unique(labels))
  
    # Create empty centroid array
    centroids = np.zeros((n_clusters, data.shape[1]))
    
    # Get indices of each cluster 
    for i in range(n_clusters):
        indices = np.where(labels == i)[0]
        
        # Calculate mean of each cluster and store it as centroid
        centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
    
    return centroids

def bcss_tss_ratio(X, cluster_centers):

    # Computing Within Cluster Sum of Squares (WCSS)
    wcss = np.sum([np.nansum((x - cluster_centers[i]) ** 2) for i in range(len(cluster_centers)) for x in X[labels == i]])

    # Computing Between Cluster Sum of Squares (BCSS)
    bcss = np.sum([np.nansum((x - center) ** 2) for center in cluster_centers for x in X])

    # Computing Total Sum of Squares (TSS)
    tss = np.sum([np.nansum((x - np.nanmean(X)) ** 2) for x in X])

    # Computing the Ratio of WCSS to TSS
    wcss_tss_ratio = wcss/tss

    # Computing the Ratio of BCSS to TSS
    bcss_tss_ratio = bcss/tss

    #return wcss_tss_ratio, bcss_tss_ratio
    return bcss_tss_ratio


In [ ]:


label_sets = []

chosen = pd.DataFrame(cosine_scores)

for e in np.array([1,2,3,5,8,13])/10:
    for m in np.array([1,2,3,5,8,13,21,34,55,89,144]):
        #print(e)
        model_DB = DBSCAN(eps = e, min_samples = m, metric = 'euclidean').fit(chosen)
        labels = model_DB.labels_
        centroids = derive_centroids(chosen,labels)
        label_sets.append([e,m,len(np.unique(labels)),labels,centroids,bcss_tss_ratio(chosen,centroids)])
        
#sizes = [len(np.unique(l)) for l in label_sets]
#pd.DataFrame(sizes).hist()        

results = pd.DataFrame(label_sets)
results.columns = ['e','m','clusters','labels','centroids','ratio']
results.query('ratio > 0').sort_values('clusters',ascending=True)

C:\Users\User\AppData\Local\Temp\ipykernel_19284\1002961604.py:13: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\Temp\ipykernel_19284\1002961604.py:13: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\Temp\ipykernel_19284\1002961604.py:13: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\Temp\ipykernel_19284\1002961604.py:13: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\Temp\ipykernel_19284\1002961604.py:13: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\Temp\ipykernel_19284\1002961604.py:13: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\Temp\i

,e,m,clusters,labels,centroids,ratio
57,1.3,3,2,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[[0.07390175759792328, -0.0022267112508416176,...",250.069166
56,1.3,2,9,"[0, 0, 1, 2, -1, -1, -1, -1, -1, -1, -1, -1, -...","[[0.7631099820137024, 0.7631101012229919, 0.11...",2000.017389
55,1.3,1,241,"[0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, ...","[[0.7631099820137024, 0.7631101012229919, 0.11...",60249.946573
0,0.1,1,250,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.9999998807907104, 0.5262200832366943, 0.15...",62500.033448
11,0.2,1,250,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.9999998807907104, 0.5262200832366943, 0.15...",62500.033448
22,0.3,1,250,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.9999998807907104, 0.5262200832366943, 0.15...",62500.033448
33,0.5,1,250,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.9999998807907104, 0.5262200832366943, 0.15...",62500.033448
44,0.8,1,250,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.9999998807907104, 0.5262200832366943, 0.15...",62500.033448


In [ ]:
"""
questions = pd.DataFrame(docs)
questions['cluster'] = results.iloc[58]['labels']
"""
chosen = 56
results_df = pd.DataFrame(questions,columns=['question'])
results_df['cluster'] = results.iloc[chosen]['labels']


In [ ]:
results_df

,question,cluster
0,How many photos does the CD-ROM XA hold?,0
1,How many minutes does a Super Audio CD hold?,0
2,How much did the percentage of households with...,1
3,Who was Victoria's eighth child?,2
4,"Article IV, Section Three allows Congress to d...",-1
...,...,...
245,What term is used by the UNO to divide groups ...,-1
246,What species group are annelids' muscle contro...,-1
247,On What date was Pope Paul VI born?,6
248,Which ingredient gives beer its flavor?,-1


cluster:  0
['How many photos does the CD-ROM XA hold?'
 'How many minutes does a Super Audio CD hold?']
cluster:  1
['How much bandwidth did YouTube consume in 2014?'
 'How much did the percentage of households with access to the internet decrease between 2003 and 2013?']
cluster:  2
["Who was Victoria's eighth child? "
 'Who did Victoria blame for her poor relationship with her mother?']
cluster:  3
['What measures the amount of light entering the eye?'
 'Which devices do not produce light by luminescence?']
cluster:  4
['In medical diagnosis, what is pain considered?'
 'What is pain medicine often under disciplines like physiatry, neurology, etc?'
 'What is the term for nociceptors which respond to more than one type of stimuli?']
cluster:  5
['Who is the federal government in Germany governed by?'
 'What is the Swedish term for government?']
cluster:  6
['Who was required to sign a decree declaring Paul VI a saint?'
 'On What date was Pope Paul VI born?']
cluster:  7
['What is neut

In [ ]:
"""
#from nltk.corpus import wordnet as wn
#from nltk.corpus import wordnet_ic

dog=wn.synsets('dog', pos=wn.NOUN)[0] #get the first noun synonym of the word "dog"
print(dog)
cat=wn.synsets('cat', pos=wn.NOUN)[0]
rose=wn.synsets('rose', pos=wn.NOUN)[0]
flower=wn.synsets('flower', pos=wn.NOUN)[0]

brown_ic = wordnet_ic.ic('ic-brown.dat') #load the brown corpus to compute the IC

rose.res_similarity(flower, brown_ic)
rose.res_similarity(dog, brown_ic)
cat.res_similarity(dog, brown_ic)
"""

In [ ]:
#contexts = pd.read_csv('contexts.csv',index_col=0)

In [ ]:
cluster_topics = []
import numpy as np
for c in np.array(range(0,len(np.unique(questions_df['cluster'])),1)):
    print(c)
    docs = questions_df[questions_df['cluster']==c]['question'].values

    vocab = createVocab_lemma(docs)

    #Compute document term matrix as well idf for each term 

    termDict={}

    docsTFMat = np.zeros((len(docs),len(vocab)))

    docsIdfMat = np.zeros((len(vocab),len(docs)))

    docTermDf = pd.DataFrame(docsTFMat ,columns=sorted(vocab.keys()))
    docCount=0
    for doc in docs:
        doc= doc.translate(str.maketrans('', '', string.punctuation))
        word_synsets = [wordnet.synsets(word) for word in word_tokenize(doc.lower())]
        for word_synset in word_synsets:
            if (word_synset):
                for lemma in word_synset[0].lemmas():
                    if(lemma.name() in vocab.keys()):
                        docTermDf[lemma.name()][docCount] = docTermDf[lemma.name()][docCount] +1

        docCount = docCount +1

    feature_names = sorted(vocab.keys())
    docList=np.array(range(0,len(docs),1))

    #Computed idf for each word in vocab
    idfDict={}

    for column in docTermDf.columns:
        idfDict[column]= np.log((len(docs) +1 )/(1+ (docTermDf[column] != 0).sum()))+1

    #compute tf.idf matrix
    docsTfIdfMat = np.zeros((len(docs),len(vocab)))
    docTfIdfDf = pd.DataFrame(docsTfIdfMat ,columns=sorted(vocab.keys()))

    docCount = 0
    for doc in docs:
        for key in idfDict.keys():
            docTfIdfDf[key][docCount] = docTermDf[key][docCount] * idfDict[key]
        docCount = docCount +1 

    skDocsTfIdfdf = pd.DataFrame(docTfIdfDf, index=sorted(docList), columns=feature_names)
    cluster_topics.append(find_most_common_words(skDocsTfIdfdf,.5))
    """
    model = Top2Vec(subset.flatten(), embedding_model='universal-sentence-encoder')
    topic_sizes, topic_nums = model.get_topic_sizes()
    topic_words, word_scores, topic_nums = model.get_topics(model.get_num_topics())
    
    print(subset)
    print(topic_words)
    cluster_topics.append([subset,topic_sizes,topic_words, word_scores, topic_nums])
    """

0
1
2
3
4
5
6
7


In [ ]:

count = results.iloc[chosen]['clusters']
#set_ = questions.query('cluster >= 0').sort_values(by='cluster')
questions_df = results_df.query('cluster >= 0').sort_values(by='cluster')
for c in np.array(range(0,count-1,1)):
    print('cluster: ', c)
    #print(set_[set_['cluster']==c]['question'].values)
    print(questions_df[questions_df['cluster']==c]['question'].values)

In [ ]:
for c in np.array(range(0,len(np.unique(questions_df['cluster'])),1)):
    print(c)
    print(cluster_topics[c])
    #print(questions[c])
    docs = questions_df[questions_df['cluster']==c][['question']].values
    print(docs)

0
hold       2.000000
many       2.000000
audio      1.405465
minutes    1.405465
super      1.405465
dtype: float64
[['How many photos does the CD-ROM XA hold?']
 ['How many minutes does a Super Audio CD hold?']]
1
much          2.000000
access        1.405465
bandwidth     1.405465
between       1.405465
consume       1.405465
decrease      1.405465
percentage    1.405465
in            1.405465
internet      1.405465
dtype: float64
[['How much bandwidth did YouTube consume in 2014?']
 ['How much did the percentage of households with access to the internet decrease between 2003 and 2013?']]
2
blame           1.405465
child           1.405465
eighth          1.405465
mother          1.405465
poor            1.405465
relationship    1.405465
dtype: float64
[["Who was Victoria's eighth child? "]
 ['Who did Victoria blame for her poor relationship with her mother?']]
3
light    2.0
dtype: float64
[['What measures the amount of light entering the eye?']
 ['Which devices do not produce ligh